# Imports

In [1]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from functools import partial
from rpy2.ipython import html

# Notebook settings

In [2]:
%load_ext rpy2.ipython
html.html_rdataframe = partial(html.html_rdataframe, table_class = "docutils")
rpy2.ipython.html.init_printing()
pandas2ri.activate()

# R Packages

In [3]:
importr("easyPubMed")
importr("tidyverse")

rpy2.robjects.packages.Package as a <module 'tidyverse'>

# Mine Data from PubMed

In [4]:
%%R
# Define a function that uses a query to mine PubMed publications
getAbstracts = function(query){
  # Get entrez_ids
  get_entrez_ids = get_pubmed_ids(query)
  # Get abstracts XML format
  abstractsXML = fetch_pubmed_data(pubmed_id_list = get_entrez_ids, retstart = 0, retmax = 5000, format = "xml", encoding = "UTF-8")
  # Get abstracts reformatted into a dataframe
  PubMed_df = table_articles_byAuth(pubmed_data = abstractsXML,
                                max_chars = 1000000,
                                included_authors = 'last',
                                encoding = "UTF-8"
                                )
  # Merge tite with abstract
  PubMed_df$title_abstract = paste(PubMed_df$title, PubMed_df$abstract, sep = " ")
  # Only keep inportant columns
  data_out = PubMed_df %>% 
    select(contains(c("pmid", "title_abstract")))
}

In [5]:
%%R
# Define the query
query = '("software"[MeSH Terms] OR "programming language"[All Fields]) AND ("metabolomics"[MeSH Terms] OR "metabolomics"[All Fields] OR "metabolonics"[All Fields] OR "metabolomic"[All Fields])'

# Call the function
minedText = getAbstracts(query=query)

R[write to console]: Processing PubMed data 
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to c

In [6]:
# Check that we have the title+abstracts as a column and the pmid as a column
%R head(minedText)

,pmid,title_abstract
1,33777285,A framework for stability-based module detecti...
2,33607938,CANTARE: finding and visualizing network-based...
3,33467846,A Practical Guide to Metabolomics Software Dev...
4,33467662,Phytochemical Profiling and Quality Control of...
5,33465307,DaDIA: Hybridizing Data-Dependent and Data-Ind...
6,33417772,Affine Transformation of Negative Values for N...


In [7]:
# Save the data
%R write.csv(minedText, file="Data/minedText.csv", row.names = F)